In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
mnist = keras.datasets.mnist

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train.shape

(60000, 28, 28)

# 정규화

In [5]:
#X_train , X_test = X_train / 255.0 , X_test / 255.0

In [6]:
sc = MinMaxScaler()

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

In [8]:
model = Sequential()

model.add(Flatten(input_shape = (28,28))) # 낮은 차원으로 풀어주는 Flatten

model.add(Dense(512, activation='relu'))

model.add(Dense(512, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
Total params: 664,576
Trainable params: 664,576
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
  56/1875 [..............................] - ETA: 14s - loss: 18.4447 - accuracy: 0.7215

KeyboardInterrupt: 

In [ ]:
model.save('model4.h5')

# 아래는 숫자 분류 exe 실행

In [22]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import tensorflow as tf
class MyApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.image = QImage(QSize(400, 400), QImage.Format_RGB32)
        self.image.fill(Qt.white)
        self.drawing = False
        self.brush_size = 30
        self.brush_color = Qt.black
        self.last_point = QPoint()
        self.loaded_model = None
        self.initUI()
    def initUI(self):
        menubar = self.menuBar()
        menubar.setNativeMenuBar(False)
        filemenu = menubar.addMenu('File')
        load_model_action = QAction('Load model', self)
        load_model_action.setShortcut('Ctrl+L')
        load_model_action.triggered.connect(self.load_model)
        save_action = QAction('Save', self)
        save_action.setShortcut('Ctrl+S')
        save_action.triggered.connect(self.save)
        clear_action = QAction('Clear', self)
        clear_action.setShortcut('Ctrl+C')
        clear_action.triggered.connect(self.clear)
        filemenu.addAction(load_model_action)
        filemenu.addAction(save_action)
        filemenu.addAction(clear_action)
        self.statusbar = self.statusBar()
        self.setWindowTitle('MNIST Classifier')
        self.setGeometry(300, 300, 400, 400)
        self.show()
    def paintEvent(self, e):
        canvas = QPainter(self)
        canvas.drawImage(self.rect(), self.image, self.image.rect())
    def mousePressEvent(self, e):
        if e.button() == Qt.LeftButton:
            self.drawing = True
            self.last_point = e.pos()
    def mouseMoveEvent(self, e):
        if (e.buttons() & Qt.LeftButton) & self.drawing:
            painter = QPainter(self.image)
            painter.setPen(QPen(self.brush_color, self.brush_size, Qt.SolidLine, Qt.RoundCap))
            painter.drawLine(self.last_point, e.pos())
            self.last_point = e.pos()
            self.update()
    def mouseReleaseEvent(self, e):
        if e.button() == Qt.LeftButton:
            self.drawing = False
            arr = np.zeros((28, 28))
            for i in range(28):
                for j in range(28):
                    arr[j, i] = 1 - self.image.scaled(28, 28).pixelColor(i, j).getRgb()[0] / 255.0
            arr = arr.reshape(-1, 28, 28)
            if self.loaded_model:
                pred = self.loaded_model.predict(arr)[0]
                pred_num = str(np.argmax(pred))
                self.statusbar.showMessage('숫자 ' + pred_num + '입니다.')
    def load_model(self):
        fname, _ = QFileDialog.getOpenFileName(self, 'Load Model', '')
        if fname:
            self.loaded_model = tf.keras.models.load_model(fname)
            self.statusbar.showMessage('Model loaded.')
    def save(self):
        fpath, _ = QFileDialog.getSaveFileName(self, 'Save Image', '', "PNG(*.png);;JPEG(*.jpg *.jpeg);;All Files(*.*) ")
        if fpath:
            self.image.scaled(28, 28).save(fpath)
    def clear(self):
        self.image.fill(Qt.white)
        self.update()
        self.statusbar.clearMessage()
        

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\isl12\anaconda3\envs\py39tf2\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 3시

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
# Raw Data Loading
df = pd.read_csv('./mnist_train.csv')

In [13]:
df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.describe()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
count,60000.000000,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,4.453933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,2.889270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


In [15]:
X_data = df.drop('label', axis =1 )
y_data = df['label']

In [16]:
sc = MinMaxScaler()
sc.fit(X_data)
X_norm_data = sc.transform(X_data)

#sc.fit_transform(X_data)

In [17]:
X_train, X_test, y_train, y_test =  train_test_split(X_norm_data, y_data, test_size=0.2)

In [18]:
model = Sequential()

model.add(Flatten(input_shape = (784,)))   # Input layer

model.add(Dense(units = 10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [25]:
from tensorflow.keras.optimizers import SGD

model.compile(optimizer=SGD(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
history = model.fit(X_train, y_train, epochs = 100, verbose=1 , validation_split= 0.2 )

Epoch 1/100
1200/1200 [==============================] - 3s 2ms/step - loss: 1.9710 - accuracy: 0.3801 - val_loss: 1.5649 - val_accuracy: 0.6639
Epoch 2/100
1200/1200 [==============================] - 2s 2ms/step - loss: 1.3348 - accuracy: 0.7352 - val_loss: 1.1609 - val_accuracy: 0.7753
Epoch 3/100
1200/1200 [==============================] - 3s 2ms/step - loss: 1.0443 - accuracy: 0.7969 - val_loss: 0.9591 - val_accuracy: 0.8074
Epoch 4/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.8881 - accuracy: 0.8202 - val_loss: 0.8409 - val_accuracy: 0.8224
Epoch 5/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.7913 - accuracy: 0.8336 - val_loss: 0.7634 - val_accuracy: 0.8330
Epoch 6/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.7252 - accuracy: 0.8423 - val_loss: 0.7078 - val_accuracy: 0.8396
Epoch 7/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.6769 - accuracy: 0.8489 - val_loss: 0.6663 - val_ac

1200/1200 [==============================] - 3s 2ms/step - loss: 0.3664 - accuracy: 0.9006 - val_loss: 0.3780 - val_accuracy: 0.8944
Epoch 58/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3651 - accuracy: 0.9008 - val_loss: 0.3767 - val_accuracy: 0.8942
Epoch 59/100
1200/1200 [==============================] - 3s 2ms/step - loss: 0.3639 - accuracy: 0.9013 - val_loss: 0.3757 - val_accuracy: 0.8945
Epoch 60/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3628 - accuracy: 0.9015 - val_loss: 0.3745 - val_accuracy: 0.8946
Epoch 61/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3616 - accuracy: 0.9017 - val_loss: 0.3734 - val_accuracy: 0.8947
Epoch 62/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3605 - accuracy: 0.9020 - val_loss: 0.3723 - val_accuracy: 0.8954
Epoch 63/100
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3595 - accuracy: 0.9024 - val_loss: 0.3713 - val_accuracy

In [50]:
loss, accuracy =  model.evaluate(X_test, y_test)

375/375 [==============================] - 1s 1ms/step - loss: 0.3611 - accuracy: 0.8992


In [51]:
round(loss * 100, 2)

36.11

In [52]:
round(accuracy * 100, 2)

89.92